In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib



# class LinearRegression:
#     def __init__(self, n_features: int): # initalize parameters
#         np.random.seed(10)
#         self.W = np.random.randn(n_features, 1) ## randomly initialise weight
#         self.b = np.random.randn(1) ## randomly initialise bias
        
#     def __call__(self, X): # how do we calculate output from an input in our model?
#         ypred = np.dot(X, self.W) + self.b
#         return ypred # return prediction
    
#     def update_params(self, W, b):
#         self.W = W ## set this instance's weights to the new weight value passed to the function
#         self.b = b ## do the same for the bias]

#     def plot_predictions(y_pred, y_true):
#         samples = len(y_pred)
#         plt.figure()
#         plt.scatter(np.arange(samples), y_pred, c='r', label='predictions')
#         plt.scatter(np.arange(samples), y_true, c='b', label='true labels', marker='x')
#         plt.legend()
#         plt.xlabel('Sample numbers')
#         plt.ylabel('Values')
#         plt.show()
    
#     def mean_squared_error(y_pred, y_true):  # define our criterion (loss function)
#         errors = y_pred - y_true  ## calculate errors
#         squared_errors = errors ** 2  ## square errors
#         return np.mean(squared_errors)
    
#     def minimize_loss(X_train, y_train):
#         X_with_bias = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
#         optimal_w = np.matmul(
#             np.linalg.inv(np.matmul(X_with_bias.T, X_with_bias)),
# #             np.matmul(X_with_bias.T, y_train),
#         )
        # return optimal_w[1:], optimal_w[0]


products = pd.read_pickle('cleaned_products.pkl')

lin_regression = LinearRegression()
X_train =
y_train = 
X_test =
y_test
lin_regression.fit(X, y)
y_pred = lin_regression.predict(X_test)

metrics.metrics.mean_squared_error(y_test, y_pred)

# save model weights
# joblib.dump(lin_regression, "linear_regression.joblib")

,id,product_name,product_description,page_id,city_town,county,price_gbp,sub_cat_0,sub_cat_1,sub_cat_2,sub_cat_3
1,243809c0-9cfc-4486-ad12-3b7a16605ba9,Mirror wall art,"Mirror wall art. Posted by Nisha in Dining, Li...",1426704584,Wokingham,Berkshire,5.0,Home & Garden,"Dining, Living Room Furniture","Mirrors, Clocks & Ornaments",None
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,Stainless Steel Food Steamer,Morphy Richard’s (model no 48755)Stainless ste...,1426704579,Inverness,Highland,20.0,Home & Garden,Other Household Goods,None,None
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,Sun loungers,I have 2 of these - collection only as I don’t...,1426704576,Skegness,Lincolnshire,20.0,Home & Garden,Garden & Patio,Outdoor Settings & Furniture,None
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,Great reclaimed army ammunition box used as co...,1426704575,Radstock,Somerset,115.0,Home & Garden,"Dining, Living Room Furniture",Other,None
5,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost,New Design Shannon Corner sofa 5 Seater Avail...,1426704570,Delph,Manchester,450.0,Home & Garden,"Dining, Living Room Furniture","Sofas, Armchairs & Suites",Private
